In [1]:
from google import genai
import json

client = genai.Client()

In [6]:
json_file_path = 'batchProcessing_file_promptTemplate3.json'

print(f"Uploading JSONL file: {json_file_path}")
batch_input_file = client.files.upload(
    file=json_file_path
    )
print(f"Uploaded JSONL file: {batch_input_file.name}")

print("\nCreating batch job...")
batch_job_from_file = client.batches.create(
    model="gemini-2.5-flash",
    src=batch_input_file.name,
    config={
        'display_name': 'batchProcessing_file_promptTemplate3',
    }
)
print(f"Created batch job from file: {batch_job_from_file.name}")
print("You can now monitor the job status using its name.")

Uploading JSONL file: batchProcessing_file_promptTemplate3.json
Uploaded JSONL file: files/9dolr0uy2hvk

Creating batch job...
Created batch job from file: batches/i519yu8mjbyuqq8lyhnp13b0fies15v1x4bz
You can now monitor the job status using its name.


In [2]:
completed_states = set([
    'JOB_STATE_SUCCEEDED',
    'JOB_STATE_FAILED',
    'JOB_STATE_CANCELLED',
    'JOB_STATE_EXPIRED',
])

print("Listing recent batch jobs:\n")

# Note: The list API currently doesn't return inlined_responses.
# As a workaround,you can make a `get` call for inline jobs to see their results.
batches = client.batches.list(config={'page_size': 10})

for b in batches.page:
    print(f"Job Name: {b.name}")
    print(f"  - Display Name: {b.display_name}")
    print(f"  - State: {b.state.name}")
    print(f"  - Create Time: {b.create_time.strftime('%Y-%m-%d %H:%M:%S')}")
    if b.state.name in completed_states:
        print(f"  - End Time: {b.end_time.strftime('%Y-%m-%d %H:%M:%S')}")




    # Check if it was an inline job (no destination file)
    if b.dest is not None:
      if not b.dest.file_name:
        full_job = client.batches.get(name=b.name)
        if full_job.inlined_responses:
            print("  - Type: Inline ({} responses)".format(len(full_job.inlined_responses)))
      else:
          print(f"  - Type: File-based (Output: {b.dest.file_name})")

    print("-" * 20)

Listing recent batch jobs:

Job Name: batches/i519yu8mjbyuqq8lyhnp13b0fies15v1x4bz
  - Display Name: batchProcessing_file_promptTemplate3
  - State: JOB_STATE_SUCCEEDED
  - Create Time: 2025-09-17 23:53:22
  - End Time: 2025-09-18 06:17:19
  - Type: File-based (Output: files/batch-i519yu8mjbyuqq8lyhnp13b0fies15v1x4bz)
--------------------
Job Name: batches/9f8onr8qkxes8tedtd357x7k6s6ceaajw72p
  - Display Name: batchProcessing_file_promptTemplate2
  - State: JOB_STATE_SUCCEEDED
  - Create Time: 2025-09-14 20:00:36
  - End Time: 2025-09-15 04:32:14
  - Type: File-based (Output: files/batch-9f8onr8qkxes8tedtd357x7k6s6ceaajw72p)
--------------------
Job Name: batches/oob9e7t1o0crucxqe7b16aeq3k5k6gb47d5t
  - Display Name: batchProcessing_file_promptTemplate1
  - State: JOB_STATE_SUCCEEDED
  - Create Time: 2025-09-13 00:29:55
  - End Time: 2025-09-13 05:49:04
  - Type: File-based (Output: files/batch-oob9e7t1o0crucxqe7b16aeq3k5k6gb47d5t)
--------------------
Job Name: batches/11wqncexbtisoa2

In [4]:
batch_job = client.batches.get(name="batches/i519yu8mjbyuqq8lyhnp13b0fies15v1x4bz")

if batch_job.state.name == 'JOB_STATE_SUCCEEDED':
    # The output is in another file.
    result_file_name = batch_job.dest.file_name
    print(f"Results are in file: {result_file_name}")

    print("\nDownloading and parsing result file content...")
    file_content_bytes = client.files.download(file=result_file_name)
    file_content = file_content_bytes.decode('utf-8')

    pretty_file_content = ""

    #The result file is also a JSONL file. Parse and print each line.
    #for line in file_content.splitlines():
    #  if line:
    #    parsed_response = json.loads(line)
    #    response_text = parsed_response['response']['candidates'][0]['content']['parts'][0]['text']
    #    print(response_text)
    #    print(json.dumps(response_text, indent=4))


    #   pretty_file_content = pretty_file_content + json.dumps(parsed_response, indent=2) + "\n"
        #Pretty-print the JSON for readability
    #    print(json.dumps(parsed_response, indent=2))
    #    print("-" * 20)

    json_file_path = 'batch_output_promptTemplate2.json'
    print(f"\nCreating JSONL file: {json_file_path}")
    with open(json_file_path, 'w') as f:
      f.write(json.dumps(file_content) + '\n')

else:
    print(f"Job did not succeed. Final state: {batch_job.state.name}")

Results are in file: files/batch-i519yu8mjbyuqq8lyhnp13b0fies15v1x4bz


Creating JSONL file: batch_output_promptTemplate2.json
